In [ ]:
pip install sphinx

navigate to ECS_demo/

touch core.py

cut and paste all that code stuff below into core.py

cd ../docs/

sphinx-autogen index.rst

make html

you got an error! : pip install sphinx_rtd_theme

# *pythonic* package development with the trimmings

## Overview

By the grace of open-source-dev there are several free lunches you should know of:

2. [sphinx](http://www.sphinx-doc.org/en/master/)
    1. sphinx can be a bit [finicky](https://samnicholls.net/2016/06/15/how-to-sphinx-readthedocs/). The most important feature to introduce to you to today will be 
    2. [autodocs](https://samnicholls.net/2016/06/15/how-to-sphinx-readthedocs/) where we generate documentation from just your 
    3. [docstrings](http://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html) super cool!
1. [read the docs](https://readthedocs.org/)
    1. not only is it free, but read the docs has a magnicent protocol for turning your hard-earned digital documentation to a pdf. Possibly my favorite feature I'll mention today.
3. [travis CI](https://travis-ci.org/)
    1. "CI" stands for continuous integration. These folks provide you with a free service -- up to 1 hour of CPU time on their servers to run all of your unit tests. 
4. [pypi](https://pypi.org/)
    1. You want people using your code as fast as possible, right? 
5. [coveralls](https://coveralls.io/)
    1. how much of that passed build is covered?!

## Sphinx

1. clone this repo. 

2. cd into the main directory and checkout the directory structure with tree:

```
$ tree
.
├── ECS_demo
│   ├── __init__.py
│   ├── core.py
│   ├── data
│   │   ├── climate_sentiment_m1.h5
│   │   └── tweet_global_warming.csv
│   ├── input.py
│   ├── tests
│   │   ├── __init__.py
│   │   └── test_ECS_demo.py
│   └── version.py
├── LICENSE
├── README.md
├── appveyor.yml
├── docs
│   ├── Makefile
│   ├── _static
│   ├── conf.py
│   ├── index.rst
│   └── source
│       ├── ECS_demo.core.rst
│       ├── ECS_demo.rst
│       └── ECS_demo.tests.rst
├── examples
│   ├── README.ipynb
│   └── README.txt
├── requirements.txt
└── setup.py
```

3. We're about to find out just how busy this directory structure can be with these added open source features. But for now, the main project lives under `ECS_demo/` with `tests/` and `data/` subdirectories.

4. Go ahead and inspect the contents of the core.py and test_ECS_demo.py files, in case you're interested. There's some common elements here in the package development world. `core.py` contains, well, the core code of the package. In a larger package you might have other modules living here such as `analysis.py` or `visualize.py`, depending on how you want to organize your code. For now, the `core.py` file contains four functions: `load_data, data_setup, baseline_model` and one class: `Benchmark`. You can learn more about pythonic naming conventions from the [pep8](https://www.python.org/dev/peps/pep-0008/) documentation.

5. Time to get to Sphinx! cd over to the docs directory. In this tutorial, I've setup the appropriate rst files already. I haven't had excellent luck with using sphinx-quickstart or sphinx-autogen, personally. And so I will always start with a template such as this and modify the `.rst` files as needed. Suffice to say, if you are interested in creating your documentation from scratch I found this [source](https://samnicholls.net/2016/06/15/how-to-sphinx-readthedocs/) helpful.

6. All you need to do is type `make html` in the `docs/` directory where your `Makefile` is sitting. and Sphinx will generate static html documents of your site.

```
$ tree -L 2
.
├── Makefile
├── _build
│   ├── doctrees
│   └── html
├── _static
├── conf.py
├── index.rst
└── source
    ├── ECS_demo.core.rst
    ├── ECS_demo.rst
    └── ECS_demo.tests.rst
```

7. Use your preferred browser to checkout your site: `open _build/html/index.html` 

![](demo1.png)

8. If you navigate to the API you'll see how Sphinx autmoatically formats your docstrings for you, super neat!

![](demo2.png)


## Github Pages

1. At this point we're ready to make our documentation live. Personally, my favorite method while I'm still developing a project is Github pages. This is because github allows us to directly host our statically generated files with Sphinx. As your code grows up, you'll want to migrate to something more robust like readthedocs.io. The downside of doing this initially is that readthedocs compiles your website as you push to github and little changes in your code structure or prerequisites can break the build. It's just easier to put this work off until you're at version 0.0.1.

1. In your browser, navigate to the settings folder for your cloned github project and scroll down until you see Github Pages. Change the Source option to master branch /docs folder and hit save.

2. Now in your local repo we're going to do a bit of a juggling act. We've been working in docs/ directory we'll want to move this sphinx stuff to its own home and make sure our statically generated files live here. So in your main directory this would like:
```
$ mkdir sphinx
$ mv docs/* sphinx/
$ mv sphinx/_build/html/* docs/
$ tree -L 1 docs/
├── docs
│   ├── _sources
│   ├── _static
│   ├── genindex.html
│   ├── index.html
│   ├── objects.inv
│   ├── py-modindex.html
│   ├── search.html
│   ├── searchindex.js
│   └── source
```
3. by default, github uses jekyll to build its' sites. We'll want to turn this feature off since our site is already built. In the `docs/` directory simply type `touch .nojekyll`

4. add/commit/push your changes. Your project is now live under the url (yourgihtubname).io/ECS

## Travis CI

1. If you haven't already, navigate over to travis-ci.org and create an account with them, then import the ECS project

2. We're going to use travis to run our unit tests. Good coding practice dictates that we also check our code for readability. This is done using pep8. Locally you'll add the following into a Makefile in the main directory:
```
flake8: 
        @if command -v flake8 > /dev/null; then \ 
                echo "Running flake8"; \ 
                flake8 flake8 --ignore N802,N806,F401 `find . -name \*.py | grep -v setup.py | grep -v /docs/ | grep -v /sphinx/`; \ 
        else \ 
                echo "flake8 not found, please install it!"; \ 
                exit 1; \ 
        fi; 
        @echo "flake8 passed" 
```
Basically, we're asking flake8 to run some but not all, tests on some but not all, files. 

at the end of that file we'll also add:
```
test: 
        py.test 
```
You can now run `make flake8` and `make test` and see that your package passes your unittests, locally.

To get this up and running with travis we'll need to add a .travis.

## Coveralls 

## pypi 

1. sdist vs bdist
2. GCP
3. API's (twitter for one)

Put this in core_test.py: